In [ ]:



from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    FunctionMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)
from langchain_core.output_parsers import (
    PydanticOutputParser,

)

# Deprecated!
# from langchain.chat_models import ChatOpenAI
#instead use:

from langchain_openai import ChatOpenAI

from google.colab import userdata
# os.getenv("OPENAI_API_KEY")


from langchain_community.llms import Replicate
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from typing import List, Optional
from pydantic import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
template = PromptTemplate(
    input_variables=["topic", "user_name","user_grade"],
    template="Explain {topic} to {user_name}, who is at grade level {user_grade}. Please discuss topic at grade level {user_grade} for maximum understanding."
)




In [ ]:


key = userdata.get('OPENAI_API_KEY')
import getpass
import os

# if not os.getenv("OPENAI_API_KEY"):
#     os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

llm = ChatOpenAI(model="gpt-4o-mini", api_key=key)
user = ChatOpenAI(model="gpt-4o-mini", api_key=key)

system = """You are a 1st grader named Sally. You are learning about chess, but you know next to nothing about chess and your response will reflect as such."""
user_context = ChatPromptTemplate.from_messages(
    [
        ("system", system),
    ]
)
systemb = """You are a professional Python coder and teacher. You are very good at explaining things at the level of your audience or user, never going above their heads, speaking in a manner and with terms they will understand, and ask them grade or level appropriate questions."""
ai_context = ChatPromptTemplate.from_messages(
    [
        ("system", systemb),
    ]
)

print(f"User: {system}")
print(f"AI: {systemb}")

User: You are a 1st grader named Sally. You are learning about chess, but you know next to nothing about chess and your response will reflect as such.
AI: You are a professional Python coder and teacher. You are very good at explaining things at the level of your audience or user, never going above their heads, speaking in a manner and with terms they will understand, and ask them grade or level appropriate questions.


In [ ]:
prompt = template.format(topic="Chess rules", user_name="Sally", user_grade="9th")
ai_context.append(("human", prompt))





print(f"AI: {prompt}")

AI: Explain Chess rules to Sally, who is at grade level 9th. Please discuss topic at grade level 9th for maximum understanding.


In [ ]:

# Define structured outputs
class Questions_for_user(BaseModel):
  """Questions to ask the user about the topic."""
  questions: List[str] = Field(description="List of questions to ask the user about the topic.")
  answers: List[str] = Field(description="List of answers to the questions.")

class Judgement_of_Understanding(BaseModel):
  """Judgement of the user's understanding of the topic."""
  # questions: List[str] = Field(description="List of questions to ask the user about the topic.")
  # answers: List[str] = Field(description="List of answers to the questions.")
  misunderstandings: List[str] = Field(description="List of areas where the user may need more explanation or questions.")
  rating: Optional[int] = Field(description="Rating of the user's understanding of the topic, on a scale of 1 to 10.")

response = llm.invoke(prompt)
print(f"AI: {response.content}")



AI: Certainly! Let’s break down the rules of chess in a way that a 9th grader can easily understand.

### Overview of Chess

Chess is a two-player board game played on an 8x8 grid of squares, alternating between light and dark colors. The objective of the game is to checkmate your opponent's king, which means to put the king in a position where it cannot escape capture. 

### The Board and Pieces

1. **Chessboard**: It consists of 64 squares arranged in an 8x8 grid. Each player starts with their pieces on the two closest rows to them.

2. **Pieces**: Each player has 16 pieces: 
   - **1 King**: The most important piece; you must protect it at all costs.
   - **1 Queen**: The most powerful piece; can move any number of squares in any direction.
   - **2 Rooks**: Can move any number of squares horizontally or vertically.
   - **2 Knights**: Moves in an "L" shape – two squares in one direction and then one square perpendicular.
   - **2 Bishops**: Can move any number of squares diagonally

In [ ]:
response.content

'Certainly! Let’s break down the rules of chess in a way that a 9th grader can easily understand.\n\n### Overview of Chess\n\nChess is a two-player board game played on an 8x8 grid of squares, alternating between light and dark colors. The objective of the game is to checkmate your opponent\'s king, which means to put the king in a position where it cannot escape capture. \n\n### The Board and Pieces\n\n1. **Chessboard**: It consists of 64 squares arranged in an 8x8 grid. Each player starts with their pieces on the two closest rows to them.\n\n2. **Pieces**: Each player has 16 pieces: \n   - **1 King**: The most important piece; you must protect it at all costs.\n   - **1 Queen**: The most powerful piece; can move any number of squares in any direction.\n   - **2 Rooks**: Can move any number of squares horizontally or vertically.\n   - **2 Knights**: Moves in an "L" shape – two squares in one direction and then one square perpendicular.\n   - **2 Bishops**: Can move any number of squar

In [ ]:


# Define templates
question_template = PromptTemplate(
    input_variables=["user_response", "topic", "ai_response"],
    template="Please write several questions about topic {topic} to ask the user to test their understanding of the topic, based on their response {user_response} and the AI's response {ai_response} prior to that."
)

template_b = PromptTemplate(
    input_variables=["user_response", "topic", "ai_response", "answers_to_questions", "questions"],
    template="This is the user's response to {ai_response} on topic {topic}, and these are the answers to questions that the user was asked about the topic: {answers_to_questions}. Please determine if the user has any questions, misunderstandings, or areas where they may need more explanation, based on their response {user_response}."
)


# Example data
user_response = "So, in chess, you win if you take the opponents queen?"
print(f"User: {user_response}")
ai_initial_response ="Allow me to ask you some questions to gauge your level of knowledge, so I know how best to teach you."
print(f"AI: {ai_initial_response}")
# print(ai_initial_response)
topic = "Chess rules"
ai_response = response.content
questions_prompt = question_template.format(user_response=user_response, topic=topic, ai_response=ai_response)
ai_context.append(("human", user_response))
ai_context.append(("ai", questions_prompt))

# print(questions_prompt)


structured_llm = llm.with_structured_output(Questions_for_user)

ai_context.append(AIMessage(content=ai_response))
ai_context.append(
        HumanMessage(content=questions_prompt))

user_context.append(HumanMessage(content=ai_response))
user_context.append(AIMessage(content=user_response))
user_context.append(HumanMessage(content=ai_initial_response))
user_context.append(AIMessage(content="Sure, go ahead."))

questions = structured_llm.invoke(questions_prompt)

answers_to_questions = []
for question in questions.questions:
    print(f"AI: {question}")
    user_context.append(("human", question))
    ai_context.append(("ai", question))
    answer = user.invoke(question).content
    answers_to_questions.append(answer)
    user_context.append(("ai", answer))
    ai_context.append(("human", answer))
    print(f"User: {answer}")


structured_llm_2 = llm.with_structured_output(Judgement_of_Understanding)


# Format list inputs as strings
formatted_questions = "\n".join(f"- {q}" for q in questions)
formatted_answers = "\n".join(f"- {a}" for a in answers_to_questions)

# Render prompt
formatted_prompt = template_b.format(
    user_response=user_response,
    topic=topic,
    ai_response=ai_response,
    questions=formatted_questions,
    answers_to_questions=formatted_answers
)


judgement = structured_llm_2.invoke(formatted_prompt)
print(f"AI Evaluation")
for misunderstanding in judgement.misunderstandings:
  print(f"AI: {misunderstanding}")
print(f"AI: {judgement.rating}")



User: So, in chess, you win if you take the opponents queen?
AI: Allow me to ask you some questions to gauge your level of knowledge, so I know how best to teach you.
AI: What is the primary objective of a chess game?
User: The primary objective of a chess game is to checkmate your opponent's king. This means putting the king in a position where it is under threat of capture (in "check") and cannot escape from capture on the next move. The game can also end in a draw under certain conditions, but the main goal is to achieve checkmate.
AI: Can you explain what checkmate means?
User: Checkmate is a term used in chess that indicates a situation in which a player's king is in a position to be captured (in check) and there is no legal move the player can make to escape this threat. When a player's king is checkmated, that player loses the game. The term can also be used metaphorically in other contexts to describe a decisive defeat or overcoming of an opponent's strategy. In chess, checkmat